# Table of Contents
 <p>

In [1]:
# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# change default style figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12
%watermark -a 'Ethen' -d -t -v -p numpy,pandas,sklearn,matplotlib

Ethen 2017-12-26 10:42:36 

CPython 3.5.2
IPython 6.2.1

numpy 1.13.3
pandas 0.21.1
sklearn 0.19.1
matplotlib 2.1.0


If you do not have these constraints, do not use Bayesian optimization.

- f is a black box function, with no closed form nor gradients.
- f is expensive to evaluate.
- You only have noisy observations of f.

Bayesian optimization in layman terms: We evaluate a few points and fit a regression model to it. After fitting this regression model, we can use it to predict where the minimum. Finally, we go to that point and evaluate our function. We can repeat the steps we just mentioned until we run out of budget.

tree-based models for high dimension??

One of the key problems with black box optimization is that we can never know with 100% certainty what our function’s best input is; to do so would mean testing an often-infinite number of input values. This means we have to draw the line somewhere, since we can’t test an infinite number

In [4]:
?RandomForestClassifier

In [10]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html
X, y = load_digits(n_class = 10, return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.25, random_state = 0)

def report(results):
    """report best scores and corresponding parameters"""
    print('Best score obtained: {0}'.format(results.best_score_))
    print('Parameters:')
    for param, value in results.best_params_.items():
        print('\t{}: {}'.format(param, value))

# log-uniform: understand as search over p = exp(x) by varying x
rf = RandomForestClassifier()
search_spaces = {'max_depth': Integer(3, 15)}
# Real(1e-6, 1e+6, prior = 'log-uniform')
# Categorical(['linear', 'poly', 'rbf'])
bayes_search = BayesSearchCV(
    estimator = rf,
    search_spaces = search_spaces,
    cv = 5,
    n_iter = 3,
    return_train_score = False)
bayes_search.fit(X_train, y_train)

print("val score: %s" % bayes_search.best_score_)
print("test score: %s" % bayes_search.score(X_test, y_test))

val score: 0.929472902747
test score: 0.944444444444


In [11]:
bayes_search.best_params_

{'max_depth': 13}

https://dark-element.com/2016/10/10/bayesian-optimization-of-black-box-functions/